## Tokenizador

In [5]:
import json
import re
import nltk
from nltk import word_tokenize

In [4]:
data = []
with open('dump_small_clean.jsonln', 'r', encoding="utf8") as file:
    for line in file:
        data.append(json.loads(line))

In [7]:
texto = data[0]['body']
tokens = word_tokenize(texto)
# tokens

In [8]:
tokens = [w for w in tokens if re.fullmatch('\w+', w)]
# tokens

In [10]:
def minusculas(tokens):
    return [token.lower() for token in tokens]

def remove_digitos(tokens):
    return [token for token in tokens if re.fullmatch('[^\d]*', token)]

def pega_palavras(tokens):
    return [token for token in tokens if re.fullmatch('\w+', token)]
    
def limpa_tokens(tokens):
    tokens = minusculas(tokens)
    tokens = remove_digitos(tokens)
    tokens = pega_palavras(tokens)    
    return tokens

In [11]:
from tqdm import tqdm

all_words = []
for item in tqdm(data):
    texto = item['body']
    tokens = word_tokenize(texto)
    tokens = limpa_tokens(tokens)
    all_words += tokens

100%|██████████| 11225/11225 [01:47<00:00, 104.27it/s]


In [ ]:
len(all_words)

In [17]:
def stopwords(all_words):
    stop = nltk.corpus.stopwords.words('portuguese')
    stop += nltk.corpus.stopwords.words('english')
    return [w for w in all_words if w not in stop]

# stopwords(all_words)

In [18]:
from collections import Counter
word_counts = Counter(stopwords(all_words_clean))
word_counts_list = list(word_counts.items())
word_counts_list_sorted = sorted(word_counts_list, key=lambda x: (-x[1], x[0]))
#word_counts_list_sorted

In [19]:
# from nltk import word_tokenize
# from tqdm import tqdm

# all_words_per_doc = []
# for item in tqdm(data):
#     texto = item['body']
#     tokens = word_tokenize(texto)
#     tokens = limpa_tokens(tokens)
#     all_words_per_doc.append(tokens)

In [24]:
# frequência
idf = dict([(n[0], np.log(N / n[1])) for n in doc_counts_list_sorted])

In [20]:
vocab =  word_counts_list_sorted[:3000]
vocab = dict(vocab)

In [22]:
LOWERCASE = [chr(x) for x in range(ord('a'), ord('z') + 1)]
#UPPERCASE = [chr(x) for x in range(ord('A'), ord('Z') + 1)]

#https://www.ascii-codes.com/cp860.html
LOWERCASE_OTHERS = ['ç', 'á', 'â', 'ã', 'à', 'é', 'í', 'ó', 'ú', 'ê','î', 'ô', 'û', 'õ']  # etc.
#UPPERCASE_OTHERS = [x.upper() for x in LOWERCASE_OTHERS]
LETTERS = LOWERCASE + LOWERCASE_OTHERS 

In [23]:
def edit1(text):
    words = []
    
    # Fase 1: as remoçoes.
    for p in range(len(text)):
        new_word = text[:p] + text[p + 1:]
        if len(new_word) > 0:
            words.append(new_word)
        
    # Fase 2: as adições.
    for p in range(len(text) + 1):
        for c in LETTERS:
            new_word = text[:p] + c + text[p:]
            words.append(new_word)
    
    # Fase 3: as substituições.
    for p in range(len(text)):
        orig_c = text[p]
        for c in LETTERS:
            if orig_c != c:
                new_word = text[:p] + c + text[p + 1:]
                words.append(new_word)
    
    return set(words)

def edit2(text):
    words1 = edit1(text)
    words2 = set()
    for w in words1:
        candidate_words2 = edit1(w)
        candidate_words2 -= words1
        words2.update(candidate_words2)
    words2 -= set([text])
    return words2


In [24]:
word = 'pars'
if word in vocab:
    candidatos = [word]
else:
    candidatos = []
candidatos += \
    [w for w in edit1(word) if w in vocab] \
    + [w for w in edit2(word) if w in vocab] \
    + [word]